Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1435]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import randint, random, choice, uniform
import lab9_lib

fitness = lab9_lib.make_problem(2)

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")


print(fitness.calls)

In [1436]:
POPULATION_SIZE = 200
OFFSPRING_SIZE = 50
TOURNAMENT_SIZE = 10
MUTATION_PROBABILITY = .15

SEQUENCE_LENGTH = 1000

In [1437]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

### Selection

In [1438]:
class Tournament():
    def __init__(self, size):
        self.size = size

    def select(self, pop):
        pool = [choice(pop) for _ in range(self.size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion
    
class TournamentEpsilon():
    def __init__(self, size, scheduler):
        self.size = size
        self.scheduler = scheduler
    
    def select(self, pop):
        if random() > self.scheduler.value():
            pool = [choice(pop) for _ in range(self.size)]
            champion = max(pool, key=lambda i: i.fitness)
            return champion
        else:
            return choice(pop)
        
def tweak(ind: Individual):
    obj = copy(ind)
    pos = randint(0, len(obj.genotype) - 1)
    pos2 = randint(0, len(obj.genotype) - 1)


    obj.genotype[pos] = not obj.genotype[pos]
    obj.genotype[pos2] = not obj.genotype[pos2]

    obj.fitness = None

    return obj     

def hill_climbing(ind: Individual, n_iter = 10):
    s = copy(ind)

    for _ in range(n_iter):
        r = tweak(s)
        if fitness(r.genotype) > fitness(s.genotype):
            s = r
    
    return s

### Mutation

In [1439]:
class Scheduler():
    def __init__(self, alpha, decay_rate = 0):
        self.rate = decay_rate
        self.current_value = alpha
    
    def mutation_prob():
        pass

class ConstantScheduler(Scheduler):
    def value(self):
        return self.current_value

class ExponentialDecayScheduler(Scheduler):
    def value(self):
        return_value = self.current_value
        self.current_value = self.current_value * (1 - self.rate)
       
        return return_value
    
class BitFlip():
    def __init__(self, sequence_length, scheduler, num_flips = 1):
        self.sequence_length = sequence_length
        self.scheduler = scheduler
        self.num_flips = num_flips
    
    def mutate(self, ind: Individual) -> Individual:
        offspring = copy(ind)

        if random() < self.scheduler.value():
            
            for i in range(self.num_flips):
                pos = randint(0, self.sequence_length-1)
                offspring.genotype[pos] = not offspring.genotype[pos]
                #offspring.genotype[pos] = choice([True, False])


            offspring.fitness = None
        
        return offspring



### Recombination

In [1440]:
class OnePointCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length

    def recombine(self,ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.sequence_length-1)
        offspring1 = Individual(fitness=None,
                            genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        offspring2 = Individual(fitness=None,
                            genotype=ind2.genotype[:cut_point] + ind1.genotype[cut_point:])

        assert len(offspring1.genotype) == self.sequence_length
        assert len(offspring1.genotype) == self.sequence_length

        return offspring1, offspring2

class UniformCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length
    
    def recombine(self, ind1: Individual, ind2: Individual) -> Individual:
        rvector1 = [random() for _ in range(self.sequence_length)]
        rvector2 = [random() for _ in range(self.sequence_length)]

        g1 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector1)]
        g2 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector2)]

        o1 = Individual(fitness = None, genotype=g1)
        o2 = Individual(fitness = None, genotype=g2)

        return o1, o2



In [1441]:
class EASimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.offspring_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)
            print(f'generation:{g}_fitness:{self.population[0].fitness}')

            if self.population[0].fitness == 1:
                break

        return record
    
class HybridSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in range(len(offspring)):
                offspring[i] = hill_climbing(offspring[i])
                offspring[i].fitness = self.fitness_eval(offspring[i].genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

        return record
    

class AdaptiveSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.mutation = mutation
        self.selection = selection
        self.select_parent = lambda pop : self.selection.select(pop)
        self.mutate = lambda ind : self.mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []
        explorative_control = 0
        best = 0

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                o1 = self.mutate(o1)
                o2 = self.mutate(o2)

                offspring.append(o1)
                offspring.append(o2)

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if int(1000* self.population[0].fitness) > int(1000 * best):
                best = self.population[0].fitness
                explorative_control = 0
            else:
                explorative_control += 1

                if explorative_control >= 10:
                    self.mutation.scheduler.current_value = 1
                    self.selection.scheduler.current_value = 1

                    explorative_control = 0

                    if self.population[0].fitness < 0.99:
                        newpop = population = [
                                                Individual(
                                                genotype=[choice((True, False)) for _ in range(SEQUENCE_LENGTH)],
                                                fitness=None,
                                            )
                                            for _ in range(POPULATION_SIZE)
                                        ]
                        for i in range(len(newpop)):
                            newpop[i].fitness = self.fitness_eval(newpop[i].genotype)

                        self.population = [p1 if random() < 0.6 else p2 for p1,p2 in zip(self.population, newpop)]
                    
                
            print(f'generation:{g}_explorecontrol:{explorative_control}_rate{self.selection.scheduler.current_value}_fitness:{self.population[0].fitness}')
            
            if self.population[0].fitness == 1:
                break

        return record
    



### Initialize Population

In [1442]:
population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(SEQUENCE_LENGTH)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]



### Experiment

#### instance 1

In [1443]:
fitness = lab9_lib.make_problem(1)

In [1444]:
for i in population:
    i.fitness = fitness(i.genotype)

In [1445]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.00001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE , scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.00001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim1 = EASimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim1.set_inital_pop(population)

record = sim1.simulate()

generation:0_fitness:0.539
generation:1_fitness:0.543
generation:2_fitness:0.543
generation:3_fitness:0.544
generation:4_fitness:0.547
generation:5_fitness:0.547
generation:6_fitness:0.559
generation:7_fitness:0.559
generation:8_fitness:0.561
generation:9_fitness:0.561
generation:10_fitness:0.568


generation:11_fitness:0.568
generation:12_fitness:0.568
generation:13_fitness:0.569
generation:14_fitness:0.573
generation:15_fitness:0.578
generation:16_fitness:0.579
generation:17_fitness:0.584
generation:18_fitness:0.584
generation:19_fitness:0.584
generation:20_fitness:0.584
generation:21_fitness:0.586
generation:22_fitness:0.586
generation:23_fitness:0.601
generation:24_fitness:0.601
generation:25_fitness:0.601
generation:26_fitness:0.601
generation:27_fitness:0.601
generation:28_fitness:0.601
generation:29_fitness:0.601
generation:30_fitness:0.601
generation:31_fitness:0.614
generation:32_fitness:0.614
generation:33_fitness:0.614
generation:34_fitness:0.614
generation:35_fitness:0.617
generation:36_fitness:0.617
generation:37_fitness:0.617
generation:38_fitness:0.627
generation:39_fitness:0.628
generation:40_fitness:0.628
generation:41_fitness:0.628
generation:42_fitness:0.636
generation:43_fitness:0.638
generation:44_fitness:0.638
generation:45_fitness:0.644
generation:46_fitnes

In [1446]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 19450


#### instance 2

In [1454]:
fitness = lab9_lib.make_problem(2)

In [1455]:
for i in population:
    i.fitness = fitness(i.genotype)

In [1456]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.0001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim2 = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim2.set_inital_pop(population)

record = sim2.simulate()

generation:0_explorecontrol:0_rate0.980197693043223_fitness:0.53
generation:1_explorecontrol:1_rate0.9607875174472562_fitness:0.53
generation:2_explorecontrol:0_rate0.9417617081065252_fitness:0.532
generation:3_explorecontrol:1_rate0.9231126536824614_fitness:0.532
generation:4_explorecontrol:0_rate0.9048328935585562_fitness:0.534
generation:5_explorecontrol:0_rate0.886915114855721_fitness:0.548
generation:6_explorecontrol:0_rate0.8693521495067424_fitness:0.55
generation:7_explorecontrol:1_rate0.8521369713886755_fitness:0.55
generation:8_explorecontrol:2_rate0.8352626935120188_fitness:0.55
generation:9_explorecontrol:0_rate0.8187225652655491_fitness:0.554
generation:10_explorecontrol:1_rate0.8025099697157205_fitness:0.554
generation:11_explorecontrol:2_rate0.7866184209595359_fitness:0.554
generation:12_explorecontrol:0_rate0.7710415615298397_fitness:0.566
generation:13_explorecontrol:1_rate0.7557731598519929_fitness:0.566
generation:14_explorecontrol:2_rate0.74080710775091_fitness:0.566

In [1457]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 193850


#### instance 3

In [1464]:
fitness = lab9_lib.make_problem(5)

In [1465]:
for i in population:
    i.fitness = fitness(i.genotype)

In [1466]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.00001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE , scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.00001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim3 = EASimulation(population_size=POPULATION_SIZE , offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim3.set_inital_pop(population)

record = sim3.simulate()

generation:0_fitness:0.30649
generation:1_fitness:0.30649
generation:2_fitness:0.30649
generation:3_fitness:0.30649
generation:4_fitness:0.30649
generation:5_fitness:0.30649
generation:6_fitness:0.30649
generation:7_fitness:0.30649
generation:8_fitness:0.31547000000000003
generation:9_fitness:0.31547000000000003
generation:10_fitness:0.31547000000000003
generation:11_fitness:0.31547000000000003
generation:12_fitness:0.31547000000000003
generation:13_fitness:0.3747
generation:14_fitness:0.3747
generation:15_fitness:0.3747
generation:16_fitness:0.3747
generation:17_fitness:0.3747
generation:18_fitness:0.3747
generation:19_fitness:0.3747
generation:20_fitness:0.3747
generation:21_fitness:0.3747
generation:22_fitness:0.3747
generation:23_fitness:0.3747
generation:24_fitness:0.3747
generation:25_fitness:0.3747
generation:26_fitness:0.3747
generation:27_fitness:0.3747
generation:28_fitness:0.3747
generation:29_fitness:0.3747
generation:30_fitness:0.3747
generation:31_fitness:0.3747
generatio

In [1467]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 0.52
num calls: 50900


#### instance 4

In [1468]:
fitness = lab9_lib.make_problem(10)

In [1469]:
for i in population:
    i.fitness = fitness(i.genotype)

In [1470]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.0001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler, num_flips=10)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim4 = EASimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim4.set_inital_pop(population)

record = sim4.simulate()

generation:0_fitness:0.25433467
generation:1_fitness:0.25433467
generation:2_fitness:0.25433467
generation:3_fitness:0.25433467
generation:4_fitness:0.25433467
generation:5_fitness:0.25433467
generation:6_fitness:0.25433467
generation:7_fitness:0.25433467
generation:8_fitness:0.25433467
generation:9_fitness:0.25433467
generation:10_fitness:0.25433467
generation:11_fitness:0.25433467
generation:12_fitness:0.25433467
generation:13_fitness:0.25433467
generation:14_fitness:0.25433467
generation:15_fitness:0.25433467
generation:16_fitness:0.25433467
generation:17_fitness:0.25433467
generation:18_fitness:0.25433467
generation:19_fitness:0.25433467
generation:20_fitness:0.25433467
generation:21_fitness:0.25433467
generation:22_fitness:0.25433467
generation:23_fitness:0.25433467
generation:24_fitness:0.25433467
generation:25_fitness:0.25433467
generation:26_fitness:0.25433467
generation:27_fitness:0.25433467
generation:28_fitness:0.25433467
generation:29_fitness:0.25433467
generation:30_fitnes

In [1471]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 0.35882
num calls: 50950
